In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, roc_curve, auc, recall_score, precision_score, roc_auc_score
from sklearn.model_selection import KFold, cross_val_score

y_tr = pd.read_csv('tox21_labels_train.csv.gz', index_col=0, compression="gzip")
y_te = pd.read_csv('tox21_labels_test.csv.gz', index_col=0, compression="gzip")
x_tr = pd.read_csv('tox21_dense_train.csv.gz', index_col=0, compression="gzip")
x_te = pd.read_csv('tox21_dense_test.csv.gz', index_col=0, compression="gzip")

In [2]:
rows_tr = np.isfinite(y_tr['SR.p53']).values
rows_te = np.isfinite(y_te['SR.p53']).values
y_train = y_tr['SR.p53'][rows_tr]
x_train = x_tr[rows_tr]
y_test = y_te['SR.p53'][rows_te]
x_test = x_te[rows_te]

In [3]:
y_train

NCGC00166288-01    1.0
NCGC00185752-01    1.0
NCGC00094121-01    0.0
NCGC00241107-01    1.0
NCGC00094586-01    0.0
NCGC00256289-01    0.0
NCGC00094893-01    1.0
NCGC00181129-01    0.0
NCGC00168772-01    1.0
NCGC00257257-01    1.0
NCGC00159428-02    1.0
NCGC00254654-01    1.0
NCGC00254030-01    1.0
NCGC00091563-01    0.0
NCGC00015195-03    1.0
NCGC00024762-02    0.0
NCGC00254272-01    1.0
NCGC00093644-01    1.0
NCGC00255759-01    1.0
NCGC00259570-01    1.0
NCGC00255770-01    1.0
NCGC00016748-01    1.0
NCGC00181159-01    1.0
NCGC00259338-01    1.0
NCGC00015889-05    1.0
NCGC00255326-01    1.0
NCGC00255493-01    1.0
NCGC00024379-04    0.0
NCGC00164562-01    1.0
NCGC00260509-01    1.0
                  ... 
NCGC00261020-01    0.0
NCGC00261951-01    0.0
NCGC00260695-01    0.0
NCGC00261723-01    0.0
NCGC00261216-01    1.0
NCGC00260894-01    0.0
NCGC00260885-01    0.0
NCGC00261323-01    0.0
NCGC00261288-01    0.0
NCGC00260744-01    0.0
NCGC00261490-01    0.0
NCGC00261059-01    0.0
NCGC0026095

In [4]:
x_train

,AW,AWeight,Arto,BertzCT,Chi0,Chi1,Chi10,Chi2,Chi3,Chi3c,...,W3D,W3DH,WNSA1,WNSA2,WNSA3,WPSA1,WPSA2,WPSA3,grav,rygr
NCGC00166288-01,7.692313e+06,18.126,2.154,2.989,17.225,12.313,1.087,10.483,9.550,1.211,...,1994.795,7065.4030,120.5720,-536.244,-20.892,363.315,1615.771,70.3990,73.7670,4.837
NCGC00185752-01,6.171000e+00,12.982,2.205,3.064,28.001,18.493,3.129,18.231,14.857,4.560,...,4706.930,19850.1320,173.4330,-1657.758,-68.337,516.621,4938.176,82.5130,99.5360,5.001
NCGC00094121-01,1.428572e+07,14.113,1.857,2.425,9.845,6.109,0.101,5.521,3.858,1.016,...,319.287,2111.4430,66.1010,-204.517,-12.768,146.310,452.669,23.7140,28.1120,2.635
NCGC00241107-01,1.024183e+07,13.927,2.105,3.106,27.656,18.427,1.838,15.234,13.196,2.236,...,3704.816,14272.9750,208.1800,-1455.262,-38.494,366.523,2562.181,52.9150,111.6860,4.006
NCGC00094586-01,5.170000e+00,13.850,2.160,2.867,18.516,11.631,0.586,11.819,9.469,2.939,...,1459.308,4910.0530,134.2220,-814.395,-56.818,208.867,1267.198,47.2820,60.5720,3.699
NCGC00256289-01,5.555562e+06,13.493,2.056,3.072,25.535,16.722,1.098,14.992,11.964,2.436,...,2994.106,11424.7670,105.3940,-716.088,-27.113,306.822,2084.705,48.9510,105.4680,3.621
NCGC00094893-01,1.095239e+07,14.255,2.056,3.060,24.405,16.333,1.446,14.720,12.884,2.389,...,3654.343,13753.2920,161.4140,-1267.345,-53.857,418.726,3287.583,67.1150,100.2030,4.752
NCGC00181129-01,7.051000e+00,13.994,2.182,3.046,23.372,15.923,1.413,14.445,11.810,2.244,...,3078.194,10800.2520,153.5140,-1171.743,-55.653,377.771,2883.413,72.1910,85.0900,4.374
NCGC00168772-01,4.388000e+00,12.676,2.424,3.238,22.275,15.983,6.921,15.933,15.770,3.202,...,2678.294,8124.7520,157.6310,-956.568,-49.644,279.866,1698.340,53.7000,81.5630,3.943
NCGC00257257-01,6.480000e+00,14.943,2.250,3.076,22.242,15.529,2.022,14.169,12.714,2.141,...,3296.595,9601.5150,185.9020,-1148.428,-48.666,334.670,2067.457,64.1950,86.4470,5.314


In [5]:
y_test

NCGC00261900-01    0.0
NCGC00260869-01    0.0
NCGC00261776-01    0.0
NCGC00261842-01    1.0
NCGC00261662-01    1.0
NCGC00261190-01    1.0
NCGC00260731-01    1.0
NCGC00261926-01    0.0
NCGC00261579-01    1.0
NCGC00260921-01    0.0
NCGC00261119-01    0.0
NCGC00261812-01    0.0
NCGC00261544-01    0.0
NCGC00261633-01    0.0
NCGC00261395-01    0.0
NCGC00261717-01    0.0
NCGC00261424-01    1.0
NCGC00261861-01    1.0
NCGC00261924-01    0.0
NCGC00261052-01    1.0
NCGC00261683-01    1.0
NCGC00261026-01    1.0
NCGC00261057-01    0.0
NCGC00261738-01    0.0
NCGC00261348-01    0.0
NCGC00261503-01    0.0
NCGC00260693-01    0.0
NCGC00260703-01    0.0
NCGC00260716-01    0.0
NCGC00260782-01    0.0
                  ... 
NCGC00357206-01    0.0
NCGC00357250-01    0.0
NCGC00357227-01    0.0
NCGC00356997-01    0.0
NCGC00356998-01    0.0
NCGC00357282-01    0.0
NCGC00357029-01    0.0
NCGC00356948-01    0.0
NCGC00357054-01    0.0
NCGC00356956-01    0.0
NCGC00356994-01    0.0
NCGC00356949-01    0.0
NCGC0035723

In [6]:
x_test

,AW,AWeight,Arto,BertzCT,Chi0,Chi1,Chi10,Chi2,Chi3,Chi3c,...,W3D,W3DH,WNSA1,WNSA2,WNSA3,WPSA1,WPSA2,WPSA3,grav,rygr
NCGC00261900-01,2.612482e+07,12.688,2.226,3.226,37.329,25.440,3.663,24.200,20.222,4.565,...,9687.312,42351.9070,194.4440,-2518.829,-83.110,772.051,10001.075,131.6330,145.9670,5.499
NCGC00260869-01,8.333337e+06,17.500,2.167,2.923,16.353,10.872,1.193,11.116,9.279,2.693,...,1256.410,2621.8850,104.0110,-475.829,-33.456,219.411,1003.763,76.7030,76.0430,3.728
NCGC00261776-01,4.074000e+00,12.464,2.364,3.043,14.681,10.826,2.149,9.980,9.469,1.342,...,1072.430,3152.6480,93.4860,-341.628,-21.327,174.791,638.757,32.8850,45.9330,3.657
NCGC00261842-01,4.838000e+00,14.509,2.087,2.880,16.872,10.920,0.413,10.035,7.719,2.090,...,1217.075,4343.4600,134.8020,-816.522,-55.496,192.858,1168.142,33.1900,57.0650,3.635
NCGC00261662-01,1.111111e+07,13.535,2.111,2.821,11.665,8.326,0.679,7.398,6.307,0.955,...,707.586,2495.6030,108.7210,-317.857,-24.629,167.138,488.595,33.8640,35.3430,3.895
NCGC00261190-01,4.234000e+00,14.747,2.211,2.799,13.621,8.927,0.722,9.321,6.798,2.436,...,791.769,1283.6460,128.6740,-406.842,-35.550,86.222,272.626,19.4700,44.1000,3.671
NCGC00260731-01,3.958000e+00,14.892,2.125,2.665,11.544,7.686,0.326,6.845,5.464,1.099,...,524.169,1676.8000,79.0090,-227.586,-21.547,147.834,425.881,26.1260,33.0660,3.659
NCGC00261926-01,7.143000e+00,12.561,2.138,2.926,20.476,14.152,0.853,11.997,9.662,1.427,...,2037.221,8400.4740,122.0600,-682.398,-34.875,279.336,1561.598,47.6350,65.5800,3.814
NCGC00261579-01,3.713000e+00,13.777,2.118,2.683,12.414,8.151,0.342,7.209,5.741,1.218,...,575.515,2494.1540,58.8930,-233.758,-20.717,178.973,710.399,31.9430,35.7230,3.283
NCGC00260921-01,4.893000e+00,12.570,2.240,2.942,17.225,12.204,1.410,10.989,9.293,1.485,...,1388.855,3853.4160,111.3670,-505.015,-34.223,180.513,818.609,38.2380,54.5270,3.620


In [7]:
label_counts = y_train.value_counts()
active = float(label_counts[1])
inactive = float(label_counts[0])
wpos = inactive/active

In [10]:
gbm = xgb.XGBClassifier(objective='binary:logistic', scale_pos_weight = wpos).fit(x_train, y_train)
y_pred = gbm.predict(x_test)

In [11]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[452 123]
 [ 16  25]]
              precision    recall  f1-score   support

         0.0       0.97      0.79      0.87       575
         1.0       0.17      0.61      0.26        41

    accuracy                           0.77       616
   macro avg       0.57      0.70      0.57       616
weighted avg       0.91      0.77      0.83       616



In [12]:
confusion_matrix(y_test, y_pred)

array([[452, 123],
       [ 16,  25]], dtype=int64)

In [14]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.97      0.79      0.87       575
         1.0       0.17      0.61      0.26        41

    accuracy                           0.77       616
   macro avg       0.57      0.70      0.57       616
weighted avg       0.91      0.77      0.83       616

